In [21]:
import time

import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import MACCSkeys, rdFingerprintGenerator

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, recall_score
from sklearn.neural_network import MLPClassifier

import matplotlib.pyplot as plt

In [22]:
SEED = 22

In [23]:
data_df = pd.read_csv('EGFR_compounds_lipinski.csv', index_col=0)
data_df.shape

(4635, 10)

In [24]:
data_df.head()

,molecule_chembl_id,IC50,units,smiles,pIC50,molecular_weight,n_hba,n_hbd,logp,ro5_fulfilled
0,CHEMBL63786,0.003,nM,Brc1cccc(Nc2ncnc3cc4ccccc4cc23)c1,11.522879,349.021459,3,1,5.2891,True
1,CHEMBL35820,0.006,nM,CCOc1cc2ncnc(Nc3cccc(Br)c3)c2cc1OCC,11.221849,387.058239,5,1,4.9333,True
2,CHEMBL53711,0.006,nM,CN(C)c1cc2c(Nc3cccc(Br)c3)ncnc2cn1,11.221849,343.043258,5,1,3.5969,True
3,CHEMBL66031,0.008,nM,Brc1cccc(Nc2ncnc3cc4[nH]cnc4cc23)c1,11.096910,339.011957,4,2,4.0122,True
4,CHEMBL53753,0.008,nM,CNc1cc2c(Nc3cccc(Br)c3)ncnc2cn1,11.096910,329.027607,5,2,3.5726,True


In [25]:
data_df = data_df[['molecule_chembl_id', 'smiles', 'pIC50', ]]

In [26]:
data_df.head()

,molecule_chembl_id,smiles,pIC50
0,CHEMBL63786,Brc1cccc(Nc2ncnc3cc4ccccc4cc23)c1,11.522879
1,CHEMBL35820,CCOc1cc2ncnc(Nc3cccc(Br)c3)c2cc1OCC,11.221849
2,CHEMBL53711,CN(C)c1cc2c(Nc3cccc(Br)c3)ncnc2cn1,11.221849
3,CHEMBL66031,Brc1cccc(Nc2ncnc3cc4[nH]cnc4cc23)c1,11.096910
4,CHEMBL53753,CNc1cc2c(Nc3cccc(Br)c3)ncnc2cn1,11.096910


In [27]:
cut_off = 6.3

In [32]:
def set_activity(value):
    if value > cut_off:
        return 1.0
    else:
        return 0.0

In [33]:
data_df['active'] = data_df['pIC50'].apply(set_activity)

In [34]:
data_df['active'].sum(), len(data_df) - data_df['active'].sum()

(2631.0, 2004.0)

In [35]:
data_df.head()

,molecule_chembl_id,smiles,pIC50,active
0,CHEMBL63786,Brc1cccc(Nc2ncnc3cc4ccccc4cc23)c1,11.522879,1.0
1,CHEMBL35820,CCOc1cc2ncnc(Nc3cccc(Br)c3)c2cc1OCC,11.221849,1.0
2,CHEMBL53711,CN(C)c1cc2c(Nc3cccc(Br)c3)ncnc2cn1,11.221849,1.0
3,CHEMBL66031,Brc1cccc(Nc2ncnc3cc4[nH]cnc4cc23)c1,11.096910,1.0
4,CHEMBL53753,CNc1cc2c(Nc3cccc(Br)c3)ncnc2cn1,11.096910,1.0


In [36]:
def smiles_to_fp(smiles, method='maccs', n_bits=2048):
    mol = Chem.MolFromSmiles(smiles)

    if method == 'maccs':
        return MACCSkeys.GenMACCSKeys(mol)
    if method == 'morgan2':
        morgan_gen  = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=n_bits)
        return morgan_gen.GetFingerprint(mol)
    if method == 'morgan3':
        morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=3, fpSize=n_bits)
        return morgan_gen.GetFingerprint(mol)
    else:
        print(f'Unkonwn mwthod: {method}. Using default fingerprint generator!')
        return MACCSkeys.GenMACCSKeys(mol)

In [37]:
compound_df = data_df.copy()

In [38]:
compound_df['fp'] = compound_df['smiles'].apply(lambda x: smiles_to_fp(x, 'maccs'))

In [41]:
len(compound_df['fp'].unique())

4635

In [42]:
def plot_roc_curves_for_models(models, test_x, test_y, save_png=False):
    """
    Helper function to plot customized roc curve.

    Parameters
    ----------
    models: dict
        Dictionary of pretrained machine learning models.
    test_x: list
        Molecular fingerprints for test set.
    test_y: list
        Associated activity labels for test set.
    save_png: bool
        Save image to disk (default = False)

    Returns
    -------
    fig:
        Figure.
    """

    fig, ax = plt.subplots()

    # Below for loop iterates through your models list
    for model in models:
        # Select the model
        ml_model = model["model"]
        # Prediction probability on test set
        test_prob = ml_model.predict_proba(test_x)[:, 1]
        # Prediction class on test set
        test_pred = ml_model.predict(test_x)
        # Compute False postive rate and True positive rate
        fpr, tpr, thresholds = roc_curve(test_y, test_prob)
        # Calculate Area under the curve to display on the plot
        auc = roc_auc_score(test_y, test_prob)
        # Plot the computed values
        ax.plot(fpr, tpr, label=(f"{model['label']} AUC area = {auc:.2f}"))

    # Custom settings for the plot
    ax.plot([0, 1], [0, 1], "r--")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.set_title("Receiver Operating Characteristic")
    ax.legend(loc="lower right")
    # Save plot
    if save_png:
        fig.savefig("roc_auc", dpi=300, bbox_inches="tight", transparent=True)
    return fig

In [43]:
def model_performance(ml_model, test_x, test_y, verbose=True):
    """
    Helper function to calculate model performance

    Parameters
    ----------
    ml_model: sklearn model object
        The machine learning model to train.
    test_x: list
        Molecular fingerprints for test set.
    test_y: list
        Associated activity labels for test set.
    verbose: bool
        Print performance measure (default = True)

    Returns
    -------
    tuple:
        Accuracy, sensitivity, specificity, auc on test set.
    """

    # Prediction probability on test set
    test_prob = ml_model.predict_proba(test_x)[:, 1]

    # Prediction class on test set
    test_pred = ml_model.predict(test_x)

    # Performance of model on test set
    accuracy = accuracy_score(test_y, test_pred)
    sens = recall_score(test_y, test_pred)
    spec = recall_score(test_y, test_pred, pos_label=0)
    auc = roc_auc_score(test_y, test_prob)

    if verbose:
        # Print performance results
        # NBVAL_CHECK_OUTPUT        print(f"Accuracy: {accuracy:.2}")
        print(f"Sensitivity: {sens:.2f}")
        print(f"Specificity: {spec:.2f}")
        print(f"AUC: {auc:.2f}")

    return accuracy, sens, spec, auc

In [44]:
def model_training_and_validation(ml_model, name, splits, verbose=True):
    x_train, x_test, y_train, y_test = splits
    ml_model.fit(x_train, y_train)
    accuracy, sens, spec, auc = model_performance(model_performance, x_test, y_test, verbose)
    return accuracy, sens, spec, auc

In [45]:
fingerprint_to_model = compound_df['fp'].to_list()
label_to_model = compound_df['active'].to_list()

In [46]:
static_train_x, static_test_x, static_train_y, static_test_y = train_test_split(
    fingerprint_to_model, label_to_model, test_size=0.2, random_state=SEED
)

In [47]:
splits = [static_train_x, static_test_x, static_train_y, static_test_y]